In [1]:
import pandas as pd
import json
from collections import defaultdict
from collections import Counter
import time 
from six.moves import urllib
import wget
import logging
logging.basicConfig(filename='facebook_test.log',level=logging.DEBUG)
import pickle as pkl
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs

In [2]:
month_conv = {1:'01', 2:'02', 3:'03', 4:'04', 5:'05', 6:'06', 7:'07', 8:'08', 9:'09', 10:'10', 11:'11', 12:'12'}
higher_date = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}

In [4]:
domains = pd.read_csv("../lite_data/domaintitles_annotated.csv", header=0)
domains.head(50)

,Unnamed: 0,domain,title_text,is_news
0,0,indiatoday.in,"Latest News, Breaking News Today - Bollywood, ...",1.0
1,3,prajavani.net,"Prajavani | ಪ್ರಜಾವಾಣಿ ತಾಜಾ ಸುದ್ದಿ, ಸಮಾಚಾರ, ವಾರ...",1.0
2,4,thehindu.com,"\nThe Hindu: Breaking News, India News, Sports...",1.0
3,6,abplive.com,"Hindi News, Breaking News in Hindi, हिंदी न्यू...",1.0
4,8,news18.com,"News, Breaking News, Latest News, News Headlin...",1.0
5,9,thedailystar.net,The Daily Star – Leading English Daily among B...,1.0
6,10,indiatimes.com,"Trending stories on Indian Lifestyle, Culture,...",1.0
7,18,amarujala.com,Hindi News (हिन्दी न्यूज़): Latest News In Hin...,1.0
8,23,washingtonpost.com,"The Washington Post: Breaking News, World, US,...",1.0
9,29,foreignpolicy.com,Foreign Policy – the Global Magazine of News a...,1.0


In [4]:
links = domains.loc[domains['is_news']==1]['domain'].tolist()
print(links[:5])
print(len(links))

['indiatoday.in', 'prajavani.net', 'thehindu.com', 'abplive.com', 'news18.com']
662


In [5]:
# d = "thehindu.com"
# folderpath = "/home/phadke/ONR/ONR/big_data/Facebook_link_posts/" + d + "/"
# if not os.path.exists(folderpath):
#             os.makedirs(folderpath)


# for m in range(12, 13):
#     month = "{0:0=2d}".format(m)
#     newfile = folderpath + "2021_" + month + ".json"
#     with open(newfile, "w") as jfile:
#         for dt in range(1, 3):
#             lower_date = "2021-" + month + "-"+"{0:0=2d}".format(dt)
#             upper_date = "2021-" + month + "-"+"{0:0=2d}".format(dt+1)
#             q = 'https://api.crowdtangle.com/links?token=IOspkA7AKgiNqZPvmhGDJ7xxSJWYYErUtZCUiBOh&link=' + d + '&startDate=' + lower_date + '&endDate=' + upper_date + '&platforms=facebook&sortBy=total_interactions&count=1000'
#             response = requests.get(q)
#             q_respose = json.loads(response.content.decode('utf-8'))
#             try:
#                 postcontainer  = q_respose['result']['posts']
#             except:
#                 logstr = d + "_" + lower_date + "_" + upper_date + ":fail"
#                 logging.info(logstr)
#                 postcontainer = []

#             if len(postcontainer) > 0:
#                 for p in postcontainer:
#                     json.dump(p, jfile)
#                     #jfile.write(p)
#                     jfile.write("\n")
            
        
        
    

In [6]:
q_respose

{'status': 200,
 'notes': '',
 'result': {'posts': [{'platformId': '100044481604368_462788638547186',
    'platform': 'Facebook',
    'date': '2021-12-02 09:52:38',
    'updated': '2022-01-17 13:21:12',
    'type': 'photo',
    'message': '"This day in 2015 which changed my life for ever" ------------------- On this day 6 years ago, It was morning 3.00 to 4.00 am on December 2nd, It was raining non-stop for almost 20 hrs now. Most of the rain gauge stations in Chennai have reached 300 mm and the ones near Tambaram and Chembarabakkam had crossed 450 mm. Never ever the upstream areas like Tambaram or Chembarabakkam have received so much rains in their history. The msgs in FB received was close to 3000 nos on that night alone and i could not open few of them. Most msgs had same content from when will rains stop to, please send us boats or help us, our locality is surrounded by water upto 1st floor. we need help. My laptop had power back for few minutes, there was power cut and with intern

In [7]:
lower_date

'2021-12-02'

In [8]:
upper_date

'2021-12-03'

In [9]:
# from pytangle.api import API
# api = API(token="IOspkA7AKgiNqZPvmhGDJ7xxSJWYYErUtZCUiBOh")
# an_url = 'https://www.youtube.com/watch?v=dQw4w9WgXcQ'
# for a_post in api.links(count=5, link=an_url):#, startDate=lower_date, endDate=upper_date):
#     print(a_post)

In [10]:
'https://api.crowdtangle.com/links?token=IOspkA7AKgiNqZPvmhGDJ7xxSJWYYErUtZCUiBOh&link=' + "youtube.com" + '&startDate=' + lower_date + '&endDate=' + upper_date + '&platforms=facebook&sortBy=total_interactions'

'https://api.crowdtangle.com/links?token=IOspkA7AKgiNqZPvmhGDJ7xxSJWYYErUtZCUiBOh&link=youtube.com&startDate=2021-12-02&endDate=2021-12-03&platforms=facebook&sortBy=total_interactions'

In [8]:
import pandas as pd
import json
from collections import defaultdict
from collections import Counter
import time 
from six.moves import urllib
import wget
import logging
logging.basicConfig(filename='facebook_lp_jan18.log',level=logging.DEBUG)
import pickle as pkl
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs
import glob

month_conv = {1:'01', 2:'02', 3:'03', 4:'04', 5:'05', 6:'06', 7:'07', 8:'08', 9:'09', 10:'10', 11:'11', 12:'12'}
higher_date = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}

domains = pd.read_csv("/home/phadke/ONR/ONR/lite_data/domains_for_facebook.csv", header=0)

done_fnames = [x.replace('/home/phadke/ONR/ONR/big_data/Facebook_link_posts/', "") for x in glob.glob("/home/phadke/ONR/ONR/big_data/Facebook_link_posts/*")]

domains = domains.loc[~domains['domain'].isin(done_fnames)]

links = domains.loc[domains['is_news']==1]['domain'].tolist()

In [9]:
links

['timesofindia.com',
 'india.com',
 'ndtv.com',
 'livelaw.in',
 'timesnownews.com',
 'outlookindia.com',
 'sundayguardianlive.com',
 'opindia.com',
 'firstpost.com',
 'tv9hindi.com',
 'tfipost.com',
 'wionews.com',
 'mathrubhumi.com',
 'dinamalar.com',
 'indianarrative.com',
 'nakkheeran.in',
 'maalaimalar.com',
 'abpnadu.com',
 'bhaskarhindi.com',
 'patrika.com',
 'bhaskar.com',
 'ndtv.in',
 'sambadenglish.com',
 'altnews.in',
 'aajtak.in',
 'thewire.in',
 'etvbharat.com',
 'telegraphindia.com',
 'thequint.com',
 'ommcomnews.com',
 'odishatv.in',
 'kanaknews.com',
 'prameyanews7.com',
 'sambad.in',
 'prameya.com',
 'samajalive.in',
 'enewsinsight.com',
 'indiatv.in',
 'firstindianews.com',
 'maharashtratimes.com',
 'organiser.org',
 'tv9bharatvarsh.com',
 'theprint.in',
 'punjabkesari.com',
 'indianexpress.com',
 'jagran.com',
 'nationnews.in',
 'news24online.com',
 'nationalheraldindia.com',
 'lagatar.in',
 'newswing.com',
 'businesstoday.in',
 'cnbctv18.com',
 'asianetnews.com',
 'r

In [10]:
done_fnames

['thedailystar.net',
 'abplive.com',
 'news18.com',
 'amarujala.com',
 'business-standard.com',
 'indiatoday.in',
 'reuters.com',
 'indiatimes.com',
 'prajavani.net',
 'livemint.com',
 'thehindu.com',
 'foreignpolicy.com',
 'washingtonpost.com']